In [1]:
#PDFテキストの属性情報をもとに、テキストデータを配置
#ヘッダ・フッタ等の除外も可能
import fitz  # PyMuPDF

def extract_text_with_layout(pdf_path, page_num, y_threshold=5, 
                             header_y=None, footer_y=None, 
                             left_margin_x=None, right_margin_x=None):
    doc = fitz.open(pdf_path)
    page = doc[page_num]
    
    # ページから辞書形式でテキストを取得
    text_dict = page.get_text("dict")
    lines = []

    # すべてのspan（文字列情報）をリスト化し、bbox_y1でソート
    for block in text_dict["blocks"]:
        if "lines" in block:  # 行情報がある場合のみ
            for line in block["lines"]:
                for span in line["spans"]:
                    bbox = span["bbox"]  # 座標情報 (x0, y0, x1, y1)
                    text = span["text"]  # テキスト内容
                    font_size = span["size"]  # フォントサイズ

                    # 座標条件に基づいてフィルタリング
                    if header_y is not None and bbox[3] < header_y:  # ヘッダー領域
                        continue
                    if footer_y is not None and bbox[1] > footer_y:  # フッター領域
                        continue
                    if left_margin_x is not None and bbox[2] < left_margin_x:  # 左マージン
                        continue
                    if right_margin_x is not None and bbox[0] > right_margin_x:  # 右マージン
                        continue

                    lines.append({
                        "text": text,
                        "x0": bbox[0],
                        "x1": bbox[2],
                        "y1": bbox[3],  # bbox_y1
                        "font_size": font_size,
                        "block": block["number"]
                    })

    # bbox_y1を基準にソートし、同じ行をグループ化
    lines = sorted(lines, key=lambda x: x["y1"])
    grouped_lines = []
    current_y = None
    current_group = []

    for item in lines:
        if current_y is None or abs(item["y1"] - current_y) > y_threshold:
            # 新しい行を開始
            if current_group:
                grouped_lines.append(current_group)
            current_group = [item]
            current_y = item["y1"]
        else:
            # 同じ行に追加
            current_group.append(item)
    if current_group:
        grouped_lines.append(current_group)

    # 同じ行内のデータをx0順、block順に並べてテキストを結合
    output_text = ""
    processed_blocks = set()  # 処理済みのブロックを記録

    for group in grouped_lines:
        group = sorted(group, key=lambda x: (x["block"], x["x0"]))  # block順, x0順でソート
        line_text = ""
        previous_x1 = None  # 前回のx1値
        previous_block = None  # 前回のブロック番号

        for item in group:
            # 異なるブロックに切り替わった場合のみスペースを挿入
            if previous_x1 is not None and item["block"] != previous_block:
                gap = item["x0"] - previous_x1
#                if gap < 0:
#                    gap = 8
#                if gap > item["font_size"] / 2:  # ブロック間の隙間を考慮してスペース挿入
                    #line_text += " " * int(gap // (item["font_size"] / 2))
#                    line_text += " " * int(8)
                if gap < 0 or gap  > item["font_size"] / 2:
                    line_text += "\t"

# LINE1にブロック番号を追加
            if item["block"] not in processed_blocks:
                line_text += f"<{item['block']}>"
                processed_blocks.add(item["block"])  # このブロックを処理済みとする

            # テキストを追加（同じブロックではスペース不要）
            line_text += item["text"]
            previous_x1 = item["x1"]  # 現在のx1座標を次の基準とする
            previous_block = item["block"]  # ブロック番号を更新

        output_text += line_text.strip() + "\n"

    doc.close()
    return output_text


In [2]:
# PDFファイルとページ番号を指定して実行
# PDFファイルのパス
base_dir = "ir/2025/"
#base_dir = "ir/2024/"

#pdf_dir =  "34070_旭化成"
#pdf_filename = "24jp.pdf"
pdf_dir =  '40040_レゾナック・ホールディングス'
pdf_filename = 'pdf-sustainability-report-integratedreport-RESONAC24J_spread.pdf'
#pdf_dir =  '41880_三菱ケミカルグループ'
#pdf_filename = "23.pdf"

pdf_path = base_dir + pdf_dir + "/" + pdf_filename

page_number = 4  # 取得したいページ番号

# 除外するヘッダ・フッタ等の座標の閾値
#デフォルト値：None
header_y = None 
footer_y = None
left_margin_x = None
right_margin_x = None

#三菱ケミカル
#header_y = 24  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
#footer_y = 652  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
#left_margin_x = 174  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
#right_margin_x = 997  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

#レゾナック
#header_y = 26  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
#footer_y = 651  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
#left_margin_x = None  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
#right_margin_x = None  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

#旭化成
header_y = 37  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
footer_y = None  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
left_margin_x = None  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
right_margin_x = None  # 右マージン除外のbbox_x0閾値（範囲内の最小値）


#同じ行とみなすY値の幅
y_threshold=5
# 実行
formatted_text = extract_text_with_layout(pdf_path, 
                                          page_number, 
                                          y_threshold,
                                          header_y, 
                                          footer_y, 
                                          left_margin_x, 
                                          right_margin_x)
print(formatted_text)

# ファイルへの書き込み
output_file_path = pdf_path[:len(pdf_path)-4]  + "_formatted_page" + str(page_number) + ".txt"

with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(formatted_text)



<4>一方で従業員は自律していかないといけません。どんどん	<31>フォリオから見た「レゾナックらしさ」です。利益的な面でも、	<60>そのために、①レゾナックの戦略に適合しているか、②レゾ	<68>　例えば、AIはこの先、社会を大きく変えていくでしょう。そ
<5>素晴らしい開発をして、製品やサービスだけでなく、従業員	<32>半導体材料の素材となる樹脂を、そのまま「素材」として売る	<61>ナックのハードルレートをクリアする収益性を持っているか、	<69>のAI用半導体のパッケージは、より多くの情報を速く処理で
<6>のポテンシャルを世界へ発信し続けていくこと。この環境が	<33>のと、フィラー（粉）を混ぜて加工し半導体メーカーが求める	<62>③レゾナックがベストオーナーかどうか、という3つの視点に	<70>きるものにするため、一つの基板の上にロジックとメモリー
<7>実現すれば日本の会社の経営は絶対に変わっていくと思って	<34>「機能性を持たせた材料」にして売るのでは、利益率がまった	<63>立って、事業ポートフォリオの入れ替えを行っています。	<71>の両方を載せなければならなかったり、2.xD、3Dと呼ばれ
<8>います。遠大なる実験かもしれませんが、私はJTCのカル	<35>く違います。機能性材料にすれば、EBITDAマージンで25	<64>　無論、目指すべき事業ポートフォリオの姿は世の中の要請	<72>るようにチップを積み上げたりしなければなりません。その
<9>チャーを革新できるという実例をレゾナックでお見せしたい	<36>〜30％確保できます。	<65>によって変わっていくものです。大きな投資では見えないリ	<73>ためには材料も革新が必要です。半導体メーカー各社は競っ
<10>と思っています。	<37>　そして、もう一つ、強調しておきたい「レゾナックらしさ」が	<66>スクを過大評価し、二の足を踏んでしまいそうになりますが、	<74>てパッケージの進化を進めていますが、当社は進化に貢献す
<11>　——これが、レゾナックが社会を変えていく、私のゴールイ	<38>あります。それは、パーパス・バリューを徹底して浸透させ企	<67>前述の3つの視点から考えた、今この時点で目指すべき姿を	<75>る新材料を既に3つ提供して世界トップシェアとなっていま
